<div align="middle">
  <h1><b><i>تمرین چهارم</i></b></h1>
 </div>




In [61]:
import pyspark
from pyspark import SparkContext 
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import col ,mean ,stddev,count




<div align="right" dir="auto">
   
#  اتصال به کلاستر اسپارک 

    در این قسمت از تمرین باید به عنوان درایور یک سسشن به اسپارک بسازیم.
 </div>
  
    

In [42]:
spark_session = SparkSession.builder.appName("homework3").master("spark://spark-master:7077").config("fs.defaultFS","hdfs://namenode:9000/").getOrCreate()
spark_session


<div align="right" dir="auto">
   
#  خواندن داده 

    :در اسپارک ما ساختارهای مختلفی برای کار با داده و پخش شدن آن‌ها در شبکه داریم که به ۳ دسته تقسیم بندی میشوند 
+ RDD
+ Dataset
+ DataFrame
    
    
     برای مطالعه بیشتر به لینک زیر مراجعه کنید:
[این لینک](https://phoenixnap.com/kb/rdd-vs-dataframe-vs-dataset)

    ما در درس با ساختار RDD آشنا شدیم حال در این تمرین میخواهیم با ساختار Dataframe آشنا شویم به به کمک آن دیتا را از روی HDFS بخوانیم و روی آن فایل‌ها پردازش انجام دهیم
 
 </div>
  
    

In [47]:
df = spark_session.read.csv("/homework3/users",header=True)
df.registerTempTable("Users")
df.show()

+--------+----------+---+-----------+
|    name| last_name|age|        job|
+--------+----------+---+-----------+
|  Sandra|       Lee| 34|    Surgeon|
|   Jacob|     Adams| 40|  Developer|
|   Terry|    Morgan| 43|      Nurse|
|    Carl|  Peterson| 34|Electrician|
| Timothy|   Spanish| 45|     Waiter|
|   Peter|    Carter| 44|  Developer|
|  Edward|     Perez| 44|    Soldier|
|Brittany|     Irish| 28|     Doctor|
|Jonathan|   Ramirez| 22|    Doorman|
|   Joyce|    Miller| 33|  Scientist|
|Virginia|    Nelson| 50|  Paramedic|
|  Willie|Vietnamese| 34|  Secretary|
|    Gary|   Spanish| 25|      Nurse|
|  Elijah|   English| 24|     Waiter|
|   Bruce|   Roberts| 43|    Surgeon|
| Zachary|     Reyes| 28|  Developer|
|  Albert|    Murphy| 29|     Waiter|
|  George|   Morales| 48|      Nurse|
|    Eric|     Ortiz| 30|      Nurse|
|   David|   Spanish| 37|   Reporter|
+--------+----------+---+-----------+
only showing top 20 rows






<div align="right" dir="auto">
   
# یک مثال ...
    در اینجا به 2 روش روی این دیتا فریم‌ها کوعری میزنیم ولی با یک عملکرد و یک خروجی
    میخواهیم تعداد افرادی که بین ۱۸ تا ۲۲ سال سن دارند و دکتر هستند را بفهمیم



 </div>
  
    

In [35]:
new_df = df.filter((col("age")>=18) & (col("age") < 22) & (col("job")=="Doctor"))
new_df.count()

3541

In [39]:
new_df = spark_session.sql("SELECT *  FROM Users where (age>=18 and age <22) and (job='Doctor') " )
new_df.show()
new_df.count()

+---------+---------+---+------+
|     name|last_name|age|   job|
+---------+---------+---+------+
|   Willie|  English| 21|Doctor|
|   Lauren|  Johnson| 21|Doctor|
|Charlotte|   Morris| 20|Doctor|
|  Timothy|   Parker| 21|Doctor|
| Lawrence| Thompson| 20|Doctor|
|  Charles|    Davis| 21|Doctor|
|     Ruth|   Wright| 21|Doctor|
|  Gregory|   French| 20|Doctor|
|   Thomas| Phillips| 21|Doctor|
|Charlotte|   Rivera| 21|Doctor|
|    Grace|  English| 20|Doctor|
|  Brandon|Gutierrez| 21|Doctor|
|   Olivia|Hernandez| 20|Doctor|
|    Tyler|  English| 20|Doctor|
|     Lori|   German| 21|Doctor|
|    Megan| Peterson| 20|Doctor|
|  Heather|    Jones| 20|Doctor|
|    Larry|  English| 20|Doctor|
|   Austin|  English| 21|Doctor|
|    Jacob|  English| 21|Doctor|
+---------+---------+---+------+
only showing top 20 rows



3541


<div align="right" dir="auto">
   
# محاسبه یک کوئری
    در این قسمت  از شما انتظار می‌رود که چند کوئری زیر را بزنند و خروجی‌های آن را در فایل‌هایی به نام‌های خواسته شده 
    بریزند
+  /homework3/output/1/ ==> افرادی که Developer هستند و بالای ۳۰ سال سن دارند 
+ /homework3/output/2/ ==> همه افرادی که فامیلی آن‌ها king است
 </div>
  
    



In [27]:
# ToDo
# df... or session...



<div align="right" dir="auto">
   
# نوشتن داده و پارتیشنینگ
    زمانی که میخواهیم داده های حجیم را به صورت فایل هایی ذخیره کنیم  نمیتوانیم همه داده را در یک فایل بزرگ ذخیره کنیم به چند دلیل :
-  ذخیره سازی یک فایل بزرگ باعث ؛تنها نقطه شکست میشود؛ و با حذف آن کل داده از دست میرود
- جستجو در این یک فایل بزرگ که مرتب شده نیست دشوار و عملی نیست
- آپدیت کردن سخت تر میشود
پس تا حدودی حل این میشکل از راه حل های زیر استفاده میکنیم
# پارتیشنینگ: 
-     بر اساس یک فیلد داده هارا دسته بندی میکنیم و در دایرکتوری های مختلف میریزیم این کار را اسپارک برای ما انجام میدهد

    
 </div>
  
    



In [38]:
df.write.partitionBy("age").csv("/homework3/example/users")

In [62]:
df.groupby("job").agg(mean("age").alias("avg_age") ,stddev("age"),count("*")).sort("avg_age").show()

+-----------+------------------+-----------------+--------+
|        job|           avg_age| stddev_samp(age)|count(1)|
+-----------+------------------+-----------------+--------+
|      Nurse| 34.94920917394541|8.992485884940477|   55069|
|    Surgeon| 34.96302268085564|8.973863547576133|   55818|
|  Professor| 34.98332851672567|8.943914066757015|   55364|
|  Repairman|34.989303668841586| 8.93561491339813|   56094|
|Electrician|34.993322083797075|8.965551076717995|   55706|
|     Doctor|34.994257142339535|8.943197500964287|   55199|
|  Developer|34.998171289241085|8.955750184756367|   55777|
|  Secretary|35.003654454997736|8.938328482743362|   55275|
|  Scientist| 35.00581070107371|   8.976795164568|   55415|
|    Dentist| 35.00910070005385|8.920510936798935|   55710|
|    Soldier| 35.00969794747692| 8.94131695474283|   55785|
|    Student| 35.03476710898256|8.945817643332122|   55541|
|    Doorman| 35.03542805916629|8.933880142679095|   55775|
|  Paramedic|35.036746825182384| 8.92001

In [5]:
names = """James
Mary
Robert
Patricia
John
Jennifer
Michael
Linda
David
Elizabeth
William
Barbara
Richard
Susan
Joseph
Jessica
Thomas
Sarah
Charles
Karen
Christopher
Lisa
Daniel
Nancy
Matthew
Betty
Anthony
Margaret
Mark
Sandra
Donald
Ashley
Steven
Kimberly
Paul
Emily
Andrew
Donna
Joshua
Michelle
Kenneth
Carol
Kevin
Amanda
Brian
Dorothy
George
Melissa
Timothy
Deborah
Ronald
Stephanie
Edward
Rebecca
Jason
Sharon
Jeffrey
Laura
Ryan
Cynthia
Jacob
Kathleen
Gary
Amy
Nicholas
Angela
Eric
Shirley
Jonathan
Anna
Stephen
Brenda
Larry
Pamela
Justin
Emma
Scott
Nicole
Brandon
Helen
Benjamin
Samantha
Samuel
Katherine
Gregory
Christine
Alexander
Debra
Frank
Rachel
Patrick
Carolyn
Raymond
Janet
Jack
Catherine
Dennis
Maria
Jerry
Heather
Tyler
Diane
Aaron
Ruth
Jose
Julie
Adam
Olivia
Nathan
Joyce
Henry
Virginia
Douglas
Victoria
Zachary
Kelly
Peter
Lauren
Kyle
Christina
Ethan
Joan
Walter
Evelyn
Noah
Judith
Jeremy
Megan
Christian
Andrea
Keith
Cheryl
Roger
Hannah
Terry
Jacqueline
Gerald
Martha
Harold
Gloria
Sean
Teresa
Austin
Ann
Carl
Sara
Arthur
Madison
Lawrence
Frances
Dylan
Kathryn
Jesse
Janice
Jordan
Jean
Bryan
Abigail
Billy
Alice
Joe
Julia
Bruce
Judy
Gabriel
Sophia
Logan
Grace
Albert
Denise
Willie
Amber
Alan
Doris
Juan
Marilyn
Wayne
Danielle
Elijah
Beverly
Randy
Isabella
Roy
Theresa
Vincent
Diana
Ralph
Natalie
Eugene
Brittany
Russell
Charlotte
Bobby
Marie
Mason
Kayla
Philip
Alexis
Louis
Lori"""

In [6]:
names= names.split()

In [7]:
len(names)

200

In [8]:
jobs = """Waiter
Paramedic
Dentist
Nurse
Electrician
Doctor
Businessman
Student
Surgeon
Doorman
Secretary
Soldier
Repairman
Scientist
Reporter
Teacher
Developer
Professor"""
jobs = jobs.split()


In [9]:
jobs

['Waiter',
 'Paramedic',
 'Dentist',
 'Nurse',
 'Electrician',
 'Doctor',
 'Businessman',
 'Student',
 'Surgeon',
 'Doorman',
 'Secretary',
 'Soldier',
 'Repairman',
 'Scientist',
 'Reporter',
 'Teacher',
 'Developer',
 'Professor']

In [10]:
last_names="""Smith
English
Johnson
Scottish
Williams
Brown
Irish
Jones
Garcia
Spanish
Miller
German
French
Italian
Davis
Rodriguez
Spanish
Martinez
Spanish
Hernandez
Lopez
Spanish
Gonzales
Spanish
Wilson
Anderson
Thomas
Taylor
English
Moore
English
Jackson
English
Martin
Lee
Perez
Spanish
Thompson
White
Harris
Sanchez
Spanish
Clark
Ramirez
Spanish
Lewis
English
Robinson
Jewish
Walker
Young
Allen
King
English
Wright
English
Scott
Torres
Nguyen
Vietnamese
Hill
English
Flores
Spanish
Green
English
Adams
Nelson
Irish
Baker
English
Hall
Rivera
Spanish
Campbell
Mitchell
Carter
English
Roberts
Gomez
Spanish
Phillips
Welsh
Evans
Welsh
Turner
Diaz
Parker
English
Cruz
Spanish
Edwards
English
Collins
Reyes
Spanish
Stewart
Morris
Morales
Murphy
Irish
Cook
English
Rogers
English
Gutierrez
Spanish
Ortiz
Spanish
Morgan
Welsh
Cooper
Peterson
Bailey
Reed
English
Kelly
Irish
Howard
"""

last_names = last_names.split()

In [11]:
len(last_names)

119

In [12]:
import random

from pyspark.sql import DataFrame 

random.choice(names)


'Kathryn'

In [23]:
df = spark_context.parallelize([Row(name=random.choice(names) ,last_name=random.choice(last_names),age=random.randint(20,50) ,job=random.choice(jobs)) for i in range(1_000_000)]).toDF()
df.show()




+--------+----------+---+-----------+
|    name| last_name|age|        job|
+--------+----------+---+-----------+
|  Sandra|       Lee| 34|    Surgeon|
|   Jacob|     Adams| 40|  Developer|
|   Terry|    Morgan| 43|      Nurse|
|    Carl|  Peterson| 34|Electrician|
| Timothy|   Spanish| 45|     Waiter|
|   Peter|    Carter| 44|  Developer|
|  Edward|     Perez| 44|    Soldier|
|Brittany|     Irish| 28|     Doctor|
|Jonathan|   Ramirez| 22|    Doorman|
|   Joyce|    Miller| 33|  Scientist|
|Virginia|    Nelson| 50|  Paramedic|
|  Willie|Vietnamese| 34|  Secretary|
|    Gary|   Spanish| 25|      Nurse|
|  Elijah|   English| 24|     Waiter|
|   Bruce|   Roberts| 43|    Surgeon|
| Zachary|     Reyes| 28|  Developer|
|  Albert|    Murphy| 29|     Waiter|
|  George|   Morales| 48|      Nurse|
|    Eric|     Ortiz| 30|      Nurse|
|   David|   Spanish| 37|   Reporter|
+--------+----------+---+-----------+
only showing top 20 rows



22/12/09 09:01:09 WARN TaskSetManager: Stage 9 contains a task of very large size (8641 KiB). The maximum recommended task size is 1000 KiB.
22/12/09 09:01:09 WARN TaskSetManager: Stage 10 contains a task of very large size (8641 KiB). The maximum recommended task size is 1000 KiB.


In [25]:
df.coalesce(1).write.csv("/homework3/users",header=True)

22/12/09 09:01:23 WARN TaskSetManager: Stage 11 contains a task of very large size (34572 KiB). The maximum recommended task size is 1000 KiB.
